In [3]:

import sys
sys.path.append("/home/ahmedkashima/venv/lib/python3.12/site-packages")
!pip install gmdh
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
import gmdh
from sklearn.metrics import mean_squared_error

  Using cached gmdh-1.0.3.tar.gz (14.4 MB)
  Preparing metadata (setup.py) ... done
  Using cached docstring_inheritance-2.2.2-py3-none-any.whl.metadata (11 kB)
Using cached docstring_inheritance-2.2.2-py3-none-any.whl (24 kB)
  DEPRECATION: Building 'gmdh' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'gmdh'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [101 lines of output]
      /home/ahmedkashima/venv/lib/python3.12/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
            

Failed to build gmdh
ERROR: Failed to build installable wheels for some pyproject.toml based projects (gmdh)


ModuleNotFoundError: No module named 'gmdh'

In [ ]:
df=pd.read_csv('breast-cancer.csv',sep=",")

In [ ]:
data.head()

In [ ]:
data.shape

Проверка на пропуски в датасете

In [ ]:
data.isna().sum()

Разделим датасет на обучающую и тестовую выборку

In [ ]:
label_encoder = LabelEncoder()
df['diagnosis'] = label_encoder.fit_transform(df['diagnosis'])

In [ ]:
# Разделяем датасет на признаки и целевую переменную
X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

In [ ]:
# Нормализация данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
# Обучение модели стекинга
base_estimators = [
    ('dt', DecisionTreeClassifier(max_depth=3, random_state=42)),
    ('mlp', MLPClassifier(random_state=42))
]
stacking_model = StackingClassifier(estimators=base_estimators, final_estimator=LogisticRegression())
stacking_model.fit(X_train, y_train)

In [ ]:
# Обучение модели многослойного персептрона
mlp_model = MLPClassifier(random_state=42)
mlp_model.fit(X_train, y_train)

In [ ]:
# Обучение модели COMBI
combi_model = gmdh.Combi()
combi_model.fit(X_train, y_train, verbose=1, n_jobs=-1, test_size=0.24, limit=7,
                criterion=gmdh.Criterion(gmdh.CriterionType.REGULARITY))

In [ ]:
# Обучение модели MULTI
multi_model = gmdh.Multi()
multi_model.fit(X_train, y_train, verbose=1, n_jobs=-1, test_size=0.24, limit=0, k_best=15,
                criterion=gmdh.Criterion(gmdh.CriterionType.REGULARITY))

In [ ]:
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    # Преобразование предсказанных вероятностей в бинарные значения
    y_pred_binary = [1 if prob > 0.5 else 0 for prob in y_pred]
    accuracy = accuracy_score(y_test, y_pred_binary)
    return accuracy

In [ ]:
# Вывод результатов
print("Stacking Model Accuracy:", evaluate_model(stacking_model, X_test, y_test))
print("MLP Model Accuracy:", evaluate_model(mlp_model, X_test, y_test))
print("COMBI Model Accuracy:", evaluate_model(combi_model, X_test, y_test))
print("MULTI Model Accuracy:", evaluate_model(multi_model, X_test, y_test))